# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of Word2Vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

/Users/dennistrimarchi/anaconda3/envs/learn-env/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [2]:
raw_df = pd.read_json('News_Category_Dataset_v2.json',lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [3]:
raw_df['combined_text'] = raw_df['headline'] + ' ' + raw_df['short_description']
data = raw_df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [4]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [6]:
model = Word2Vec(data,size=100,window=5,min_count=1,workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [7]:
model.train(data,total_examples=model.corpus_count,epochs=10)

(55563615, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [8]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [9]:
wv.most_similar('Texas')

[('Pennsylvania', 0.832095205783844),
 ('Ohio', 0.812793493270874),
 ('Oregon', 0.8061274886131287),
 ('Oklahoma', 0.8026740550994873),
 ('Maryland', 0.798485279083252),
 ('Louisiana', 0.7963187098503113),
 ('Florida', 0.793173611164093),
 ('Illinois', 0.7909237146377563),
 ('Massachusetts', 0.7860420942306519),
 ('Georgia', 0.7844070792198181)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [15]:
wv.most_similar(negative=['Texas'])

[('sparks', 0.4921824336051941),
 ('roseallday', 0.43791866302490234),
 ('work-life-family-health', 0.4339059293270111),
 ('RenewDocMcStuffins', 0.4323071241378784),
 ('actonclimate', 0.418800950050354),
 ('chewiness', 0.4121505320072174),
 ('Fresh-faced', 0.41012418270111084),
 ('b.', 0.4099818170070648),
 ('bean-to-bar', 0.40995413064956665),
 ('AND', 0.4083092212677002)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [11]:
wv['Texas']

array([-1.1827875 ,  1.1248422 ,  0.6560095 , -0.27812082, -0.00484837,
        0.1371616 ,  0.2929262 ,  2.83476   , -0.9841028 ,  0.9138756 ,
       -1.3861629 ,  0.25743967,  0.28312078, -1.4235692 ,  0.6993867 ,
        0.31690028, -2.2000122 , -1.8966045 , -2.531402  ,  0.5492599 ,
       -0.05637679, -0.01261783,  0.32718664, -0.2157586 ,  0.1085865 ,
        1.8923771 ,  0.89030963, -1.581945  , -0.41339484, -3.2973928 ,
       -0.28281426,  0.17335503,  0.1245432 , -0.45162052,  1.3126112 ,
        0.22316082,  1.3315761 , -0.05321854,  0.8241836 , -1.30542   ,
       -0.3174002 , -0.5522786 , -0.5194048 , -1.6240299 , -0.5249096 ,
        0.6550727 , -1.8199404 , -2.1871326 , -0.18076749, -2.070006  ,
        0.40694982, -2.6405668 ,  1.1054443 ,  2.6776185 , -1.5359707 ,
       -1.3183165 , -0.55653644,  0.41315138, -1.1161278 ,  1.7520077 ,
        1.7843528 , -1.9466592 , -1.0424533 , -1.8204714 , -2.1358016 ,
       -1.105221  , -0.26414642, -0.60532004, -0.38444045,  1.33

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [12]:
wv.vectors

array([[-0.05810719,  0.59493613, -1.941136  , ..., -0.14306481,
        -0.87614787, -0.4101918 ],
       [ 0.0697768 ,  2.184403  , -2.798169  , ..., -0.68372697,
        -3.3561914 ,  0.24000898],
       [-0.1782154 ,  2.014996  , -2.3096285 , ...,  0.4380748 ,
        -1.7379184 , -1.2879695 ],
       ...,
       [ 0.01273434, -0.04956803,  0.08291717, ..., -0.01377183,
         0.04629448, -0.11346674],
       [ 0.04493928, -0.08044259,  0.07139303, ..., -0.02206425,
        -0.00874594, -0.08670643],
       [-0.01483792, -0.05597272,  0.0192674 , ..., -0.01555429,
         0.05564767,  0.03312906]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [14]:
wv.most_similar(positive=['king','woman'],negative=['man'])

[('title', 0.6338983178138733),
 ('queen', 0.6268595457077026),
 ('princess', 0.5909329652786255),
 ('jewel', 0.5812258720397949),
 ('revival', 0.578438401222229),
 ('coin', 0.5683972835540771),
 ('villain', 0.5643233060836792),
 ('crown', 0.5637565851211548),
 ('lover', 0.5622431039810181),
 ('creator', 0.5616716146469116)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!